In [ ]:
import os
from openpyxl import load_workbook
import pandas as pd

BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('')))
DATA_DIR = os.path.join(BASE_DIR, 'amerigo_island', 'data')

xls_filename = 'load_and_generation.xlsx'
xls_sheetname = 'Combined Data'
header_col_num = 1 

In [ ]:
wb = load_workbook(os.path.join(DATA_DIR, xls_filename), data_only=True)
ws = wb[xls_sheetname]

ws_vals = [row for row in ws.values]
ws_colnames = ws_vals[header_col_num]

load_and_gen_df = pd.DataFrame(data= ws_vals[header_col_num + 1:], columns= ws_colnames)
load_and_gen_df.columns = ['date', 'load_mw', 'gen_wind_mw', 'gen_solar_mw', 'gen_thermal_total_mw', 'gen_thermal_rice_mw', 'gen_thermal_combustion_mw']
load_and_gen_df = load_and_gen_df[['date', 'load_mw', 'gen_wind_mw', 'gen_solar_mw']]
load_and_gen_df.describe()

In [ ]:
load_and_gen_df['gen_solar_mw'] = load_and_gen_df['gen_solar_mw'].apply(lambda x: x if x > 0 else 0)
load_and_gen_df['gen_thermal_total_mw'] = load_and_gen_df['load_mw'] - load_and_gen_df['gen_wind_mw'] - load_and_gen_df['gen_solar_mw']
load_and_gen_df['gen_thermal_total_kwh'] = load_and_gen_df['gen_thermal_total_mw'] * 1000
load_and_gen_df.describe()

In [ ]:
'''
values for RICE taken from: https://www.eergy.gov/sites/prod/files/2016/09/f33/CHP-Recip%20Engines.pdf
assumed to be System #5 in paper above, for two reasons:
- System #5 has highest capacity
- System #5 has O&M similar to what is specified in assignment
'''
therm_rice_config = {
    "o_and_m" : .0105, # in cents / kwh
    "nominal_heat_rate_btu" : 9827, # in BTU / kWh
    "fuel_cost" : 2.50, # in $ / gallon
    "thermal_efficiency" : .35,
    "electric_efficiency" : .419,
    "total_efficiency" : .769,
    "column_prepend" : "rice"
}

In [ ]:
'''
values for combustion engine taken from: https://www.epa.gov/sites/production/files/2015-07/documents/catalog_of_chp_technologies_section_3._technology_characterization_-_combustion_turbines.pdf
Specifically, Table 3-2. 
Numbers reflect System #1 in paper above, for two reasons:
- System #1 has closest electric heat rate to what is specified in assignment PDF
'''
therm_combustion_config = {
    "o_and_m" : .0046, # in $ / kWh
    "nominal_heat_rate_btu" : 17680, # in BTU / kWh 
    "fuel_cost" : 2.50, # in $ / gallon
    "thermal_efficiency" : .42,
    "electric_efficiency" : .24,
    "total_efficiency" : .657,
    "column_prepend" : "combustion"
}

In [ ]:
def standardize_units_of_config(config):
    
    config["nominal_heat_rate_gal"] = config["nominal_heat_rate_btu"] / 139_000 # specified in assignment
    
    return config
    

In [ ]:
therm_combustion_config = standardize_units_of_config(therm_combustion_config)
therm_rice_config = standardize_units_of_config(therm_rice_config)

print("--- RICE config:\n{}\n".format(therm_rice_config))
print("--- Combustion config:\n{}\n".format(therm_combustion_config))

In [ ]:
def calc_addtl_cols(df, config):
    
    decorated_df = df.copy()
    efficiency_key = 'total_efficiency'
    
    col_prepend = config['column_prepend']
    o_and_m = config['o_and_m']
    nominal_heat_rate_gal = config['nominal_heat_rate_gal']
    fuel_cost = config['fuel_cost']
    efficiency = config[efficiency_key]
    
    decorated_df['{}_fuel_cost'.format(col_prepend)] = decorated_df['gen_thermal_total_kwh'] * nominal_heat_rate_gal * fuel_cost
    decorated_df['{}_o_and_m_cost'.format(col_prepend)] = decorated_df['gen_thermal_total_kwh'] * o_and_m
    decorated_df['{}_total_cost'.format(col_prepend)] = decorated_df['{}_o_and_m_cost'.format(col_prepend)] + decorated_df['{}_fuel_cost'.format(col_prepend)]
    
    return decorated_df
    

In [ ]:
load_and_gen_df_decorated = calc_addtl_cols(load_and_gen_df, therm_rice_config)
load_and_gen_df_decorated = calc_addtl_cols(load_and_gen_df_decorated, therm_combustion_config)

In [ ]:
load_and_gen_df_decorated.to_csv(os.path.join(DATA_DIR, "bau.csv"), index=False)

In [ ]:
load_and_gen_df_decorated["cheaper_gen"] = load_and_gen_df_decorated.apply(lambda x: "rice" if x['rice_total_cost'] < x['combustion_total_cost'] else "combustion", axis=1)

In [ ]:
load_and_gen_df_decorated['rice_total_cost'].describe()

In [ ]:
load_and_gen_df_decorated['rice_total_cost'].sum()

In [ ]:
load_and_gen_df_decorated['gen_thermal_total_kwh'].sum() / 8760

~$50M seems like a reasonable cost for 31MW of capacity, according to this site: https://www.power-eng.com/2017/04/18/questions-and-considerations-for-rice-generation-facilities/

This is only a quick, gut check though. Will find more resources to verify sanity of this number